# Homework 1. MNIST 데이터셋을 활용해 다중 분류 수행하기


#### 문제. 아래의 `코드`를 완성하고, 그에 대한 `분석`을 작성하시오.

1. (코드) MNIST 데이터셋 받기
   - (분석) 데이터셋에 대해 분석하기

In [5]:
# 필요 라이브러리 import
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score

In [11]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist['data'], mnist['target'].astype(np.uint8)

print("데이터의 형태:", X.shape)
print("레이블의 형태:", y.shape)

# 레이블의 값 출력
labels = np.unique(y)
print("\n레이블 값:", labels)
print("\n")

# 레이블에 해당하는 데이터 수 출력
unique, counts = np.unique(y, return_counts=True)
label_counts = dict(zip(unique, counts))

for label, count in label_counts.items():
    print(f"label {label} : {count}개")
print("\n\n")

데이터의 형태: (70000, 784)
레이블의 형태: (70000,)

레이블 값: [0 1 2 3 4 5 6 7 8 9]


label 0 : 6903개
label 1 : 7877개
label 2 : 6990개
label 3 : 7141개
label 4 : 6824개
label 5 : 6313개
label 6 : 6876개
label 7 : 7293개
label 8 : 6825개
label 9 : 6958개





In [ ]:
MNIST Dataset 분석 

MNIST 에는 70000개의 이미지 데이터가 있으며 target은 0부터 9까지의 값으로 구성되어 있다.

이미지는 28 * 28 픽셀의 흑백 손글씨 숫자로 구성되어 있으며, 확인한대로 28 * 28 = 784 개의 픽셀 데이터가 들어있다.
각 이미지는 0부터 9까지의 숫자중 하나를 나타내고, label에 속하는 데이터의 갯수는 위와 같았다.


2. (코드) 데이터셋을 train / validation / test set 으로 분할하기
   - (분석) 데이터셋을 분할한 방식에 대해 설명하고, 해당 분할 방식을 수행한 이유에 대해 설명하기

In [13]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist['data'], mnist['target'].astype(np.uint8)
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

3. (코드) 특성 스케일링을 이용한 데이터셋 전처리하기
   - (분석) 스케일링을 수행한 방식에 대해 설명하고, 해당 스케일링을 수행한 이유에 대해 설명하기

In [15]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_split)
X_val_scaled = scaler.transform(X_val_split)
X_test_scaled = scaler.transform(X_test)

4. (코드) 모델 선택하기
   - (분석) 머신러닝 모델의 후보군들에 대해 설명하고, 후보군들 중 특정 모델이 대표 모델로 선택된 이유에 대해 설명하기

1. 로지스틱 회귀 (Logistic Regression)
로지스틱 회귀는 Binary Classfication(이진 분류)를 위해 사용되지만 Softmax Regression을 사용하면 다중 클래스 분류에도 사용이 가능하다.
하지만 데이터의 선형 관계를 이용해 분석하는 로지스틱 회귀는 데이터가 비선형적인(4와 9의 유사성 등) MNIST에는 사용하기 부적합하다고 판단했다.

2. kNN, 최근접이웃 (k-Nearest Neighbors) 
새로운 데이터를 가장 가까운 k개의 이웃의 class를 참조하여 분류하는 kNN 알고리즘을 이용해 분류할 수 있다.
그러나 kNN은 새로운 데이터 포인트를 분류할 때 기존의 모든 데이터와의 거리를 계산해야 하므로, 데이터셋이 클수록 계산 비용이 급격히 증가하는 단점이 있다.
또한 단순히 이웃의 class를 기반으로 분류를 결정하기 때문에, 분류 결과에 대한 직관적인 설명이 어렵다.
Scikit learn 으로 구현한 사례가 존재한다. (https://teddylee777.github.io/scikit-learn/scikit-learn-knn/)

3. CNN, 합성곱 신경망 (Convolutional Neural Networks)
CNN은 이미지 분류 작업에서 매우 효과적인 알고리즘임을 확인할 수 있었다. 특히 데이터가 비선형적이거나 복잡한 패턴을 포함하고 있을 때 높은 성능을 보여주므로
이번 Homework에는 MNIST를 분류하는데 CNN 모델을 이용하였다.

//또한 수업시간에 다루었던 확률적 경사 하강 모델(SGD) , 랜덤포레스트 모델, 결정트리 모델을 이용해 MNIST를 분류해보고 CNN 모델과 비교해보았다.

5. (코드) 성능 평가하기
   - (분석) 최종적으로 학습된 모델을 이용해 test set 에 대한 성능을 정리하고, 이를 분석하기

In [22]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.neural_network import MLPClassifier
multilayerperceptron_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
multilayerperceptron_clf.fit(X_train_scaled,y_train_split)

multilayer_val_score = multilayerperceptron_clf.score(X_val_scaled, y_val_split)
multilayer_test_score = multilayerperceptron_clf.score(X_test_scaled, y_test)

y_pred = multilayerperceptron_clf.predict(X_test)
mtx_cf = confusion_matrix(y_test, y_pred)

In [38]:
print('Confution Matrix: \n', mtx_cf)

total_samples = mtx_cf.sum()
num_positive = mtx_cf.diagonal().sum()
num_negative = total_samples - num_positive

print('Number of Samples: \t  ',    f'{total_samples:5d}',    f'({total_samples / total_samples * 100:6.2f}%)')
print('Number of Positive Labels:', f'{num_positive:5d}', f'({num_positive / total_samples * 100:6.2f}%)')
print('Number of Negative Labels:', f'{num_negative:5d}', f'({num_negative / total_samples * 100:6.2f}%)')

Confution Matrix: 
 [[ 975    0    0    1    1    0    1    1    1    0]
 [   0 1125    3    1    0    0    1    1    4    0]
 [   5    4 1003    2    0    0    5    5    7    1]
 [   1    0    4  992    0    2    0    4    0    7]
 [   2    1    1    1  954    0    4    5    2   12]
 [   3    1    0   15    3  855    7    1    2    5]
 [   5    2    1    1    3    2  943    0    1    0]
 [   0    4    6    2    2    0    0 1006    4    4]
 [   6    1    2   10    5    4    2    4  937    3]
 [   3    2    0    5    5    4    0    5    2  983]]
Number of Samples: 	   10000 (100.00%)
Number of Positive Labels:  9773 ( 97.73%)
Number of Negative Labels:   227 (  2.27%)


In [ ]:
실행 결과 97.73%의 정확도를 보여주었다.

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
TN = mtx_cf[0, 0]
FP = mtx_cf[0, 1]
FN = mtx_cf[1, 0]
TP = mtx_cf[1, 1]

func_acc = lambda mtx_cf: (mtx_cf[0, 0] + mtx_cf[1, 1]) / (mtx_cf[0, 0] + mtx_cf[0, 1] + mtx_cf[1, 0] + mtx_cf[1, 1])
func_pre = lambda mtx_cf: (mtx_cf[1, 1]) / (mtx_cf[0, 1] + mtx_cf[1, 1])
func_rec = lambda mtx_cf: (mtx_cf[1, 1]) / (mtx_cf[1, 0] + mtx_cf[1, 1])
func_spe = lambda mtx_cf: (mtx_cf[0, 0]) / (mtx_cf[0, 0] + mtx_cf[0, 1])
func_f1 = lambda precision, recall: 2 * (precision * recall) / (precision + recall)

#교차검증을 위한 cross_validation 항목 추가
y_train_pred = cross_val_predict(multilayerperceptron_clf, X_train_scaled, y_train_split, cv=5)


acc = accuracy_score(y_train_split,y_train_pred)
prec = precision_score(y_train_split,y_train_pred,average='weighted')
recall = recall_score(y_train_split,y_train_pred,average='weighted')
f1 = f1_score(y_train_split,y_train_pred,average='weighted')

acc_ = func_acc(mtx_cf)
prec_ = func_pre(mtx_cf)
recall_ = func_rec(mtx_cf)
f1_ = func_f1(prec_, recall_)

spe_ = func_spe(mtx_cf)


print(f"Metrics:      Function ~ Confusion")
print(f"Accuracy:\t{acc:.5f} ~ {acc_:.4f}")
print(f"Precision:\t{prec:.5f} ~ {prec_:.4f}")
print(f"Recall:\t\t{recall:.5f} ~ {recall_:.4f}")
print(f"F1:\t\t{f1:.5f} - {f1_:.4f}")

Metrics:      Function ~ Confusion
Accuracy:	0.97394 ~ 1.0000
Precision:	0.97392 ~ 1.0000
Recall:		0.97394 ~ 1.0000
F1:		0.97393 - 1.0000


In [ ]:
결과 분석

Accuracy가 약 0.973 정도로 선택한 모델이 Test Set에서 올바르게 분류하고 있음을 알 수 있다.
또한 정밀도와 재현률의 결과 또한 0.973으로 높은 성능을 보여주었고,
F1 Score의 점수도 높아 정밀도와 재현율이 균형을 이루고 있음을 확인할 수 있었다.

6. (코드) 에러 분석하기
   - (분석) 최종적으로 학습된 모델의 에러를 분석하기
   - (분석) `5. 성능 평가하기` 의 분석과 연관지어 에러를 분석하고 해결책을 제안하기

참고자료